In [21]:
import pandas as pd
import sqlite3
from IPython.display import display, Markdown

conn = sqlite3.connect('ecommerce.db')
df = pd.read_csv('ecommerce_sales_clean.csv', parse_dates=['date'])
df.to_sql('sales_clean', conn, if_exists='replace', index=False)

print("✓ Database created: ecommerce.db")
print("✓ Table created: sales_clean")
print(f"✓ Rows inserted: {len(df)}")

✓ Database created: ecommerce.db
✓ Table created: sales_clean
✓ Rows inserted: 43736


In [28]:
def run_query(query_number, query, description):
    display(Markdown(f"### Query {query_number}: {description}"))
    display(Markdown(f"```sql\n{query}\n```"))
    result = pd.read_sql_query(query, conn)
    display(result)
    return result


In [29]:
query1 = """
SELECT 
    COUNT(DISTINCT transaction_id) AS total_transactions,
    COUNT(DISTINCT customer_id) AS unique_customers,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value,
    ROUND(SUM(total_revenue) * 1.0 / COUNT(DISTINCT customer_id), 2) AS revenue_per_customer
FROM sales_clean;
"""
run_query(1, query1, "Overall Business Metrics")

### Query 1: Overall Business Metrics

```sql

SELECT 
    COUNT(DISTINCT transaction_id) AS total_transactions,
    COUNT(DISTINCT customer_id) AS unique_customers,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value,
    ROUND(SUM(total_revenue) * 1.0 / COUNT(DISTINCT customer_id), 2) AS revenue_per_customer
FROM sales_clean;

```

,total_transactions,unique_customers,total_revenue,avg_order_value,revenue_per_customer
0,43736,4001,3590832.24,82.1,897.48


,total_transactions,unique_customers,total_revenue,avg_order_value,revenue_per_customer
0,43736,4001,3590832.24,82.1,897.48


In [30]:
query2 = """
SELECT 
    customer_id,
    COUNT(transaction_id) AS num_purchases,
    ROUND(SUM(total_revenue), 2) AS total_spent,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10;
"""
run_query(2, query2, "Top 10 Customers by Revenue")

### Query 2: Top 10 Customers by Revenue

```sql

SELECT 
    customer_id,
    COUNT(transaction_id) AS num_purchases,
    ROUND(SUM(total_revenue), 2) AS total_spent,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY customer_id
ORDER BY total_spent DESC
LIMIT 10;

```

,customer_id,num_purchases,total_spent,avg_order_value
0,C4288,20,2291.49,114.57
1,C3459,22,2212.18,100.55
2,C2041,20,2094.58,104.73
3,C1024,17,2032.38,119.55
4,C2944,20,2023.78,101.19
5,C3035,19,2020.18,106.33
6,C2229,17,2015.66,118.57
7,C1537,17,2013.11,118.42
8,C4920,14,1962.27,140.16
9,C4963,20,1954.80,97.74


,customer_id,num_purchases,total_spent,avg_order_value
0,C4288,20,2291.49,114.57
1,C3459,22,2212.18,100.55
2,C2041,20,2094.58,104.73
3,C1024,17,2032.38,119.55
4,C2944,20,2023.78,101.19
5,C3035,19,2020.18,106.33
6,C2229,17,2015.66,118.57
7,C1537,17,2013.11,118.42
8,C4920,14,1962.27,140.16
9,C4963,20,1954.80,97.74


In [31]:
query3 = """
WITH monthly_revenue AS (
    SELECT 
        strftime('%Y-%m', date) AS month,
        SUM(total_revenue) AS revenue
    FROM sales_clean
    GROUP BY strftime('%Y-%m', date)
)
SELECT 
    month,
    revenue,
    LAG(revenue) OVER (ORDER BY month) AS prev_month_revenue,
    ROUND(((revenue - LAG(revenue) OVER (ORDER BY month)) * 100.0 / 
           LAG(revenue) OVER (ORDER BY month)), 2) AS growth_rate_pct
FROM monthly_revenue
ORDER BY month;
"""
run_query(3, query3, "Monthly Revenue Growth Rate")

### Query 3: Monthly Revenue Growth Rate

```sql

WITH monthly_revenue AS (
    SELECT 
        strftime('%Y-%m', date) AS month,
        SUM(total_revenue) AS revenue
    FROM sales_clean
    GROUP BY strftime('%Y-%m', date)
)
SELECT 
    month,
    revenue,
    LAG(revenue) OVER (ORDER BY month) AS prev_month_revenue,
    ROUND(((revenue - LAG(revenue) OVER (ORDER BY month)) * 100.0 / 
           LAG(revenue) OVER (ORDER BY month)), 2) AS growth_rate_pct
FROM monthly_revenue
ORDER BY month;

```

,month,revenue,prev_month_revenue,growth_rate_pct
0,2022-01,135948.77,NaN,NaN
1,2022-02,126385.51,135948.77,-7.03
2,2022-03,139426.87,126385.51,10.32
3,2022-04,144076.08,139426.87,3.33
4,2022-05,135891.54,144076.08,-5.68
5,2022-06,128616.14,135891.54,-5.35
6,2022-07,136888.90,128616.14,6.43
7,2022-08,134383.71,136888.90,-1.83
8,2022-09,136698.21,134383.71,1.72
9,2022-10,139083.49,136698.21,1.74


,month,revenue,prev_month_revenue,growth_rate_pct
0,2022-01,135948.77,NaN,NaN
1,2022-02,126385.51,135948.77,-7.03
2,2022-03,139426.87,126385.51,10.32
3,2022-04,144076.08,139426.87,3.33
4,2022-05,135891.54,144076.08,-5.68
5,2022-06,128616.14,135891.54,-5.35
6,2022-07,136888.90,128616.14,6.43
7,2022-08,134383.71,136888.90,-1.83
8,2022-09,136698.21,134383.71,1.72
9,2022-10,139083.49,136698.21,1.74


In [32]:
query4 = """
SELECT 
    category,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_revenue,
    ROUND(SUM(total_revenue) * 100.0 / (SELECT SUM(total_revenue) FROM sales_clean), 2) AS revenue_share_pct,
    RANK() OVER (ORDER BY SUM(total_revenue) DESC) AS revenue_rank
FROM sales_clean
GROUP BY category
ORDER BY total_revenue DESC;
"""
run_query(4, query4, "Category Performance with Rankings")

### Query 4: Category Performance with Rankings

```sql

SELECT 
    category,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_revenue,
    ROUND(SUM(total_revenue) * 100.0 / (SELECT SUM(total_revenue) FROM sales_clean), 2) AS revenue_share_pct,
    RANK() OVER (ORDER BY SUM(total_revenue) DESC) AS revenue_rank
FROM sales_clean
GROUP BY category
ORDER BY total_revenue DESC;

```

,category,transaction_count,total_revenue,avg_revenue,revenue_share_pct,revenue_rank
0,Home,11002,1170516.02,106.39,32.60,1
1,Electronics,14736,1141369.16,77.45,31.79,2
2,Sports,10877,890580.04,81.88,24.80,3
3,Office,7121,388367.02,54.54,10.82,4


,category,transaction_count,total_revenue,avg_revenue,revenue_share_pct,revenue_rank
0,Home,11002,1170516.02,106.39,32.60,1
1,Electronics,14736,1141369.16,77.45,31.79,2
2,Sports,10877,890580.04,81.88,24.80,3
3,Office,7121,388367.02,54.54,10.82,4


In [33]:
query5 = """
SELECT 
    region,
    strftime('%Y', date) AS year,
    COUNT(*) AS transactions,
    ROUND(SUM(total_revenue), 2) AS revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY region, strftime('%Y', date)
ORDER BY region, year;
"""
run_query(5, query5, "Regional Analysis with Year-over-Year Comparison")

### Query 5: Regional Analysis with Year-over-Year Comparison

```sql

SELECT 
    region,
    strftime('%Y', date) AS year,
    COUNT(*) AS transactions,
    ROUND(SUM(total_revenue), 2) AS revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY region, strftime('%Y', date)
ORDER BY region, year;

```

,region,year,transactions,revenue,avg_order_value
0,Central,2022,4657,380960.65,81.80
1,Central,2023,4609,377773.47,81.96
2,East,2022,4257,348078.37,81.77
3,East,2023,4285,347225.99,81.03
4,North,2022,4373,364055.39,83.25
5,North,2023,4330,354079.89,81.77
6,South,2022,4316,353091.54,81.81
7,South,2023,4327,358943.37,82.95
8,West,2022,4379,357831.23,81.72
9,West,2023,4203,348792.34,82.99


,region,year,transactions,revenue,avg_order_value
0,Central,2022,4657,380960.65,81.80
1,Central,2023,4609,377773.47,81.96
2,East,2022,4257,348078.37,81.77
3,East,2023,4285,347225.99,81.03
4,North,2022,4373,364055.39,83.25
5,North,2023,4330,354079.89,81.77
6,South,2022,4316,353091.54,81.81
7,South,2023,4327,358943.37,82.95
8,West,2022,4379,357831.23,81.72
9,West,2023,4203,348792.34,82.99


In [34]:
query6 = """
SELECT 
    product_name,
    category,
    COUNT(*) AS units_sold,
    ROUND(SUM(total_amount), 2) AS product_revenue,
    ROUND(AVG(unit_price), 2) AS avg_price,
    ROUND(SUM(total_amount) * 100.0 / (SELECT SUM(total_amount) FROM sales_clean), 2) AS revenue_contribution_pct
FROM sales_clean
GROUP BY product_name, category
ORDER BY product_revenue DESC;
"""
run_query(6, query6, "Product Performance Analysis")

### Query 6: Product Performance Analysis

```sql

SELECT 
    product_name,
    category,
    COUNT(*) AS units_sold,
    ROUND(SUM(total_amount), 2) AS product_revenue,
    ROUND(AVG(unit_price), 2) AS avg_price,
    ROUND(SUM(total_amount) * 100.0 / (SELECT SUM(total_amount) FROM sales_clean), 2) AS revenue_contribution_pct
FROM sales_clean
GROUP BY product_name, category
ORDER BY product_revenue DESC;

```

,product_name,category,units_sold,product_revenue,avg_price,revenue_contribution_pct
0,Running Shoes,Sports,3629,572936.80,117.26,16.99
1,Kitchen Knife Set,Home,3675,488478.14,97.72,14.49
2,Coffee Maker,Home,3749,438820.48,88.05,13.01
3,Wireless Earbuds,Electronics,3653,384627.07,78.25,11.41
4,Bluetooth Speaker,Electronics,3684,338736.57,68.50,10.05
5,Backpack,Office,3601,283896.58,58.63,8.42
6,Laptop Stand,Electronics,3767,247146.40,48.83,7.33
7,Desk Lamp,Home,3578,188360.52,39.09,5.59
8,Yoga Mat,Sports,3653,145271.40,29.30,4.31
9,Water Bottle,Sports,3595,117904.71,24.40,3.50


,product_name,category,units_sold,product_revenue,avg_price,revenue_contribution_pct
0,Running Shoes,Sports,3629,572936.80,117.26,16.99
1,Kitchen Knife Set,Home,3675,488478.14,97.72,14.49
2,Coffee Maker,Home,3749,438820.48,88.05,13.01
3,Wireless Earbuds,Electronics,3653,384627.07,78.25,11.41
4,Bluetooth Speaker,Electronics,3684,338736.57,68.50,10.05
5,Backpack,Office,3601,283896.58,58.63,8.42
6,Laptop Stand,Electronics,3767,247146.40,48.83,7.33
7,Desk Lamp,Home,3578,188360.52,39.09,5.59
8,Yoga Mat,Sports,3653,145271.40,29.30,4.31
9,Water Bottle,Sports,3595,117904.71,24.40,3.50


In [35]:
query7 = """
WITH customer_metrics AS (
    SELECT 
        customer_id,
        COUNT(*) AS purchase_count,
        SUM(total_revenue) AS total_spent
    FROM sales_clean
    GROUP BY customer_id
)
SELECT 
    CASE 
        WHEN purchase_count >= 20 THEN 'High Frequency'
        WHEN purchase_count >= 10 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
    END AS customer_segment,
    COUNT(*) AS num_customers,
    ROUND(AVG(total_spent), 2) AS avg_lifetime_value,
    ROUND(SUM(total_spent), 2) AS segment_revenue
FROM customer_metrics
GROUP BY customer_segment
ORDER BY segment_revenue DESC;
"""
run_query(7, query7, "Customer Segmentation by Purchase Frequency")

### Query 7: Customer Segmentation by Purchase Frequency

```sql

WITH customer_metrics AS (
    SELECT 
        customer_id,
        COUNT(*) AS purchase_count,
        SUM(total_revenue) AS total_spent
    FROM sales_clean
    GROUP BY customer_id
)
SELECT 
    CASE 
        WHEN purchase_count >= 20 THEN 'High Frequency'
        WHEN purchase_count >= 10 THEN 'Medium Frequency'
        ELSE 'Low Frequency'
    END AS customer_segment,
    COUNT(*) AS num_customers,
    ROUND(AVG(total_spent), 2) AS avg_lifetime_value,
    ROUND(SUM(total_spent), 2) AS segment_revenue
FROM customer_metrics
GROUP BY customer_segment
ORDER BY segment_revenue DESC;

```

,customer_segment,num_customers,avg_lifetime_value,segment_revenue
0,Medium Frequency,2581,1044.68,2696317.91
1,Low Frequency,1396,611.20,853240.61
2,High Frequency,24,1719.74,41273.72


,customer_segment,num_customers,avg_lifetime_value,segment_revenue
0,Medium Frequency,2581,1044.68,2696317.91
1,Low Frequency,1396,611.20,853240.61
2,High Frequency,24,1719.74,41273.72


In [36]:
query8 = """
SELECT 
    CASE WHEN is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END AS period_type,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value,
    ROUND(SUM(total_revenue) * 100.0 / (SELECT SUM(total_revenue) FROM sales_clean), 2) AS revenue_share_pct
FROM sales_clean
GROUP BY is_weekend;
"""
run_query(8, query8, "Weekend vs Weekday Performance")

### Query 8: Weekend vs Weekday Performance

```sql

SELECT 
    CASE WHEN is_weekend = 1 THEN 'Weekend' ELSE 'Weekday' END AS period_type,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value,
    ROUND(SUM(total_revenue) * 100.0 / (SELECT SUM(total_revenue) FROM sales_clean), 2) AS revenue_share_pct
FROM sales_clean
GROUP BY is_weekend;

```

,period_type,transaction_count,total_revenue,avg_order_value,revenue_share_pct
0,Weekday,28608,2351052.26,82.18,65.47
1,Weekend,15128,1239779.98,81.95,34.53


,period_type,transaction_count,total_revenue,avg_order_value,revenue_share_pct
0,Weekday,28608,2351052.26,82.18,65.47
1,Weekend,15128,1239779.98,81.95,34.53


In [37]:
query9 = """
SELECT 
    payment_method,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_transaction_value,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM sales_clean
GROUP BY payment_method
ORDER BY total_revenue DESC;
"""
run_query(9, query9, "Payment Method Analysis")

### Query 9: Payment Method Analysis

```sql

SELECT 
    payment_method,
    COUNT(*) AS transaction_count,
    ROUND(SUM(total_revenue), 2) AS total_revenue,
    ROUND(AVG(total_revenue), 2) AS avg_transaction_value,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM sales_clean
GROUP BY payment_method
ORDER BY total_revenue DESC;

```

,payment_method,transaction_count,total_revenue,avg_transaction_value,unique_customers
0,Gift Card,11400,933738.73,81.91,3758
1,Credit Card,10807,891536.54,82.50,3748
2,PayPal,10754,887770.16,82.55,3699
3,Debit Card,10775,877786.81,81.47,3740


,payment_method,transaction_count,total_revenue,avg_transaction_value,unique_customers
0,Gift Card,11400,933738.73,81.91,3758
1,Credit Card,10807,891536.54,82.50,3748
2,PayPal,10754,887770.16,82.55,3699
3,Debit Card,10775,877786.81,81.47,3740


In [38]:
query10 = """
SELECT 
    quarter,
    strftime('%Y', date) AS year,
    COUNT(*) AS transactions,
    ROUND(SUM(total_revenue), 2) AS revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY quarter, year
ORDER BY year, quarter;
"""
run_query(10, query10, "Seasonal Trends Analysis")

### Query 10: Seasonal Trends Analysis

```sql

SELECT 
    quarter,
    strftime('%Y', date) AS year,
    COUNT(*) AS transactions,
    ROUND(SUM(total_revenue), 2) AS revenue,
    ROUND(AVG(total_revenue), 2) AS avg_order_value
FROM sales_clean
GROUP BY quarter, year
ORDER BY year, quarter;

```

,quarter,year,transactions,revenue,avg_order_value
0,1,2022,4872,401761.15,82.46
1,2,2022,5061,408583.76,80.73
2,3,2022,4966,407970.82,82.15
3,4,2022,7083,585701.45,82.69
4,1,2023,4993,403907.44,80.89
5,2,2023,4960,411319.89,82.93
6,3,2023,4946,408188.74,82.53
7,4,2023,6855,563398.99,82.19


,quarter,year,transactions,revenue,avg_order_value
0,1,2022,4872,401761.15,82.46
1,2,2022,5061,408583.76,80.73
2,3,2022,4966,407970.82,82.15
3,4,2022,7083,585701.45,82.69
4,1,2023,4993,403907.44,80.89
5,2,2023,4960,411319.89,82.93
6,3,2023,4946,408188.74,82.53
7,4,2023,6855,563398.99,82.19


In [39]:
query11 = """
WITH first_purchase AS (
    SELECT 
        customer_id,
        MIN(date) AS first_purchase_date
    FROM sales_clean
    GROUP BY customer_id
)
SELECT 
    strftime('%Y-%m', first_purchase_date) AS cohort_month,
    COUNT(DISTINCT customer_id) AS new_customers
FROM first_purchase
GROUP BY cohort_month
ORDER BY cohort_month;
"""
run_query(11, query11, "Cohort Analysis - Customer Acquisition")

### Query 11: Cohort Analysis - Customer Acquisition

```sql

WITH first_purchase AS (
    SELECT 
        customer_id,
        MIN(date) AS first_purchase_date
    FROM sales_clean
    GROUP BY customer_id
)
SELECT 
    strftime('%Y-%m', first_purchase_date) AS cohort_month,
    COUNT(DISTINCT customer_id) AS new_customers
FROM first_purchase
GROUP BY cohort_month
ORDER BY cohort_month;

```

,cohort_month,new_customers
0,2022-01,1360
1,2022-02,868
2,2022-03,594
3,2022-04,404
4,2022-05,273
5,2022-06,150
6,2022-07,111
7,2022-08,72
8,2022-09,61
9,2022-10,44


,cohort_month,new_customers
0,2022-01,1360
1,2022-02,868
2,2022-03,594
3,2022-04,404
4,2022-05,273
5,2022-06,150
6,2022-07,111
7,2022-08,72
8,2022-09,61
9,2022-10,44


In [40]:
query12 = """
SELECT 
    a.product_name AS product_1,
    b.product_name AS product_2,
    COUNT(*) AS times_bought_together
FROM sales_clean a
JOIN sales_clean b ON a.customer_id = b.customer_id 
    AND a.date = b.date 
    AND a.product_name < b.product_name
GROUP BY a.product_name, b.product_name
HAVING COUNT(*) > 10  -- Changed from 50 to 10 for smaller datasets
ORDER BY times_bought_together DESC
LIMIT 10;
"""
run_query(12, query12, "Product Bundle Analysis")

### Query 12: Product Bundle Analysis

```sql

SELECT 
    a.product_name AS product_1,
    b.product_name AS product_2,
    COUNT(*) AS times_bought_together
FROM sales_clean a
JOIN sales_clean b ON a.customer_id = b.customer_id 
    AND a.date = b.date 
    AND a.product_name < b.product_name
GROUP BY a.product_name, b.product_name
HAVING COUNT(*) > 10  -- Changed from 50 to 10 for smaller datasets
ORDER BY times_bought_together DESC
LIMIT 10;

```

,product_1,product_2,times_bought_together
0,Bluetooth Speaker,Desk Lamp,11


,product_1,product_2,times_bought_together
0,Bluetooth Speaker,Desk Lamp,11


In [41]:
conn.close()